# 중간고사 이후 내용 정리

## 1. Variational Bayes

> 사후 확률을 근사하는 방법

### **Motivation: Gipps Sampling**

* $z$가 확률변수, $x$가 given이라 할 때...
* 타겟 확률 $p(z_1, ..., z_d~|~x)$를 근사하기 위해 조건부 분포 $p(z_k~|~z_{-k}, x)$에서 반복적으로 샘플링을 수행하는 MCMC 방법. $z_k \sim p(z_k~|~z_{-k}, x)$
* MH 알고리즘의 특별한 형태. $(x, y) \to (x, y^*) \to (x^*, y^*)$

$$\begin{align}
(x, y) \to (x, y^*)&:~p(x, y)p(y^*~|~x) = \pi(x)k(x, y) \\
(x, y^*) \to (x, y)&:~p(x, y^*)p(y~|~x) = \pi(y)k(y, x) \\
& \Rightarrow \frac{p(x, y)}{p(y~|~x)}\frac{p(y^*~|~x)}{p(x, y^*)}
\end{align}$$

* acceptance rate가 1.0인 MH 알고리즘
* coordinate wise로 원하는 stationary distribution으로 이동
* VB의 미분 방정식 $q_j (z_j) \propto \exp (E_{-j} [\log~p(z_{1:m}, x_{1:n})])$는 Gibbs Sampling에서의 조건부 분포와 유사한 반복적 업데이트 과정

### **A. Variational Bayes and mean-field approximation**

* Mean-field approximation의 기본 가정 하에, 사후 분포 $p(z_{1:m}~|~x_{1:n})$를 독립적인 인자들의 곱으로 근사
* 기본 가정: Posterior를 근사 분포로 대체할 때, 해당 분포가 독립적인 인자들의 곱으로 분해될 수 있다고 가정. 개별적인 분포를 독립적으로 최적화할 수 있도록 만듦

$$\begin{align}
\log p(y) & \geq \text{ELBO} \\
\log p(y) - \text{ELBO} & = \text{KL}(q~||~p)
\end{align}$$

* negative ELBO에서부터 손실함수를 도출. $q_i(z_i)$는 $z_{-i}$가 주어졌을 때의 조건부 함수로 취급 (조건부 독립 가정)

$$\begin{align}
\mathcal L & = - \mathbb E_q \left[ \log \frac{\prod_{i=1}^m q_i (z_i)}{p(z_{1:m}, x_{1:n})} \right] \\
& = \int q_j(z_j) \int \prod_{k\neq j} q_k(z_k) \log p(z_{1:m}, y_{1:n})dz_{-j}dz_j - \int q_j(z_j) \int \prod_{k\neq j}q_k(z_k) \sum_{k=1}^m \log q_k(z_k)dz_{-j}dz_j
\end{align}$$

Let $\mathbb E_{-j} [\log p(z_{1:m}, y_{1:n})] = \int \prod_{k \neq j} q_k(z_k) \log p(z_{q:m}, y_{1:n})dz_{-j}$

$$\begin{align}
\mathcal L & = \int q_j(z_j) \mathbb E_{-j}[\log p(z_{1:m}, y_{1:n})]dz_j - \int q_j(z_j) \log q_j(z_j) dz_j + C
\end{align}$$

`-` 함수 미분 $\frac{\partial \mathcal L_j}{\partial q_j (z_j)}$으로부터 최적의 근사 분포 $q_j(z_j)$ 유도

$$q_j(z_j) \propto \exp(\mathbb E_{-j} [\log p(z_{1:m}, x_{1:n})])$$


* 사람이 손으로 계산해야 함. 한 개의 변수씩 기댓값을 취하면서 $q$ 분포 뽑아냄...
* 어떻게든 손으로 계산할 수 있음. 손으로 계산할 수 있는 매우 훌륭한 도구.
* bigdata에서도 잘 적용되며, 때때로 거의 유일한 대안일 수 있음.
* 분포의 평균은 잘 추정할 수 있으나, 분산은 잘 하지 못한다는 치명적인 단점. mean을 알면 좋긴 하니까...
* $p(x~|~z)$의 closed form이 존재하지 않은 경우, 파라미터 추정에 네트워크를 이용하는 등의 경우 계산이 불가능하거나 매우 어려움. 분포 자체가 매우 복잡하면 어려움. $\to$ AutoEncoder가 도입됨.

`-` Example

* Normal-Gamma Prior: 평균 $\mu$와 정밀도 $\tau$를 가지는 Normal Model에 대한 VB 적용 예시. $(\mu, \tau) ≡ z_{1:2}$

> $\tau$는 $\sigma^2$의 역수로, 값이 커질수록 분포가 밀집된다는 의미에서 정밀도(Precision)이라 함

* $q_{\mu}(\mu)$는 정규 분포를 따르며, Precision의 역수 $\tau_N^{-1}$는 $\mathbb E_{\tau} [\tau]$에 의존
* $\mu_N$은 사전과 모델 정보의 가중 평균으로 구성됨

### **B. Variation AutoEncoder**

`-` Encoder and Decoder

* Encoder: 입력 컨텍스트를 요약된 정보, 잠재 공간 Latent Space의 벡터/코드/특성으로 변환하는 과정. $X \to Z$. Unsupervised Learning이나 Dimensionality Reduction같은 간단한 것도 포함.
* Decoder: 요약된 정보를 원래의 컨텍스트로 다시 변환하는 과정. $Z \to X'$
> 요약된 정보는 Code, Latent Variable, Feature, Hidden Representation 등으로 불림

* encoding process: input $x$를 위한 $p(z~|~x)$
* decoding process: output $x$를 위한 $p(x~|~z)$
> 보통 input과 output $x$는 서로 다름

`-` Variational approach

bottleneck hidden layer를 $z$, input/output을 $x$라 할 때.

* encoding process: $p_{\phi}(z~|~x)$
* decoding process: $p_{\theta}(x~|~z)$

$$P_{\phi}(z~|~x) \log P_{\theta}(x~|~z)$$

> decoder $\log P_{\theta}(x~|~z)$는 가우시안 형태를 가정한다면 손실함수의 형태로 나타남. 이 경우 계산에 문제가 없음.
>
> encoder $P(z~|~x)$는 그 형태를 정하기 어려움. 가우시안 $q_{\phi}(z)$로 근사하는 Variational Approach가 필요. $\to$ $q$를 알고 있으므로 $z$로 인코딩한 다음 디코딩하면 됨.

* Negative ELBO Loss Function (Reconstruction Error - KL Regularization Term)
* Stochastic Gradient Variational Bayes (SGVB)
> 어차피 SGD에서 확률적인 요소가 존재하는데, 그래디언트에 확률적 요소가 더 포함된다고 해서 학습에 크게 문제될 것이 있을까? 라는 생각

* Reparametrization Trick
> $q_{\phi}(z~|~x)$에서 $z$를 샘플링하는 과정은 미분 불가능하여 Backpropagation이 불가능: sampling은 분포에서 추출하는 과정이므로, 매번 달라지고 미분 불가능. 딥러닝 학습의 핵심 요소가 작동 안함.
>
> $z$의 샘플링을 $\phi$를 통해 바로 하는 것이 아닌 별도의 변수 $\epsilon$을 샘플링함으로써 분리

$$z = \mu(x) + \sigma (x) \epsilon ~~ \text{where}~\epsilon \sim N(0, I)$$

> 모델 자체에서 확률적 요소는 제거. $A(X) + B$처럼 해석해서 그대로 딥러닝 학습.
>
> 샘플링이 아니지만 샘플링임. 가우시안도 따름

`-` 작동 과정

1. 인코더 $q_{\phi}$를 사용하여 $x$에서 latent space $z$로 변환 (분포로 변환). 인코더는 변환된 $z$의 평균과 분산을 반환
2. noise를 샘플링한 뒤, 인코더로부터 얻은 평균과 분산을 이용하여 $z$를 추출
3. 디코더에 $z$를 입력하여 재구성된 $x$로 변환.
4. negative ELBO를 계산하여 인코더 $(\phi)$와 디코더 $(\theta)$를 업데이트

`-` 장단점 및 응용 예시

* Denoising: 처음부터 잡음이 섞인 오브젝트를 삽입하면 선명해지는 효과를 볼 수 있음
* 이상치 탐색: 이상 데이터를 latent space로 변환하면 비정상적인 결과가 나올 수 있음
* 원본이 노이즈가 없는 fine image여도, 생성 결과는 뭉게질 수 있음. VAE 단독으로 파인한 이미지 생성은 불가능 -> Diffusion을 섞어야 함
* Segmentation
* 블러링, 디테일의 손실

## 2. MC Dropout

### **A. (Convolutional) Dropout**

`-` 오버피팅 방지

* Disconnection Masking: 신경망 연결을 확률적인 방식으로 끊어내는 기법
* 학습 단계에서 각 iteration마다 확률적으로 마스킹 수행 -> 활성화되는 가중치가 계속 달라짐. 배치마다 가중치 값 달라짐
* 추론 단계에서 업데이트된 마지막 step들의 가중치를 평균한 다음, $1/p$로 스케일 조정

`-` Interpretation of dropout

* 전통적인 앙상블 기법이 아님: 독립적인 모델에서 결과 산출이 아닌 상호 종속성이 존재하는 훈련 단계에서 앙상블
* 부스팅도 아님: 마스킹은 이전 스텝의 영향없이 완전 확률적으로 결정
* training step마다 다른 형태의 네트워크를 사용하는 것이라고 말할 수 있음 (not fully trained for each network)
* Gradient Flow를 부드럽게 만들어 vanishing / exploding을 완화해준다.

### **B. Bayesian DL, MC Dropout**

* 기존 Dropout 코드를 거의 그대로 가져다 사용할 수 있음.
* DNN의 필연적인 Randomness를 수학적으로 해석 $\to$ 수렴 상태인 모델을 더 훈련하여 다양한 $\hat y$을 산출하고 이를 분포로 나타내고자 했음 (해당 사실이 완전히 규명되지는 않음)
* Gradient가 0이 나와도 업데이트가 수행됨: Perturbation으로 Randomness를 더 확보...

`-` 핵심 아이디어

* 일반적인 드롭아웃을 추론 단계에서도 활성화하여 몬테 카를로 샘플링 효과를 얻는 기법
* DNN에서의 Uncertainty를 모델링할 수 있음
* Dropout Rate에 근거한 베르누이 분포와, 사전 가중치와의 결합을 Prior로 설정 $p(w_i)$

`-` Uncertainty (Mutual Information)

* 모델이 예측에 대해 얼마나 확신하는지 측정
* Naive approach: 가장 높은 확률을 1에서 빼서 불확실성으로 간주?
* Mutual Information: p(X, y)가 p(x)p(y)와 얼마나 비슷한지를 측정하는 척도 (KL)
> $H(y~|~x) - \mathbb E_q [H(y~|~x, w)]$. 즉, 모델이 가중치의 변화에 얼마나 민감한지를 측정. 작을수록 좋음. 모르겠다아
>
> 통계학에서 일반적인 MI는 클수록 좋음. y에 대한 uncertainty에서 x를 알고 난 후에 y에 대한 uncertainty를 뺀 값. 뒤쪽이 작다는 것은 uncertainty가 거의 없다는 뜻. x의 정보를 알게 됨으로써 y에 대한 불확실성이 얼마나 감소했는지를 나타냄
* Credible Interval을 잘 추정하지 못함 (근사)

## 3. Gaussian Process

### **A. Gaussian Process**

`-` 정의 및 특징

* 함수 공간에서의 확률 과정(Random Process). Random Function.
* 일반적인 베이지안 모델과 달리 함수 자체에 대한 Prior를 제공
* Continuous Stochastic Process(불가산 무한을 가산 무한으로 사영)
* 함수의 정의역 이내의 finite $(x_1, \cdots, x_N)$ 함수값의 벡터 $(f(x_1), \cdots, f(x_N)) \sim N((\mu_1, \cdots, \mu_N)^{\top}, \Sigma_N)$

* $\mu$: 랜덤하게 뿌려지는 값들 중 평균
> Prior와 비슷하여 그냥 영벡터로 뿌려도 무방
* $\boldsymbol \Sigma_N = \{ k(x_i, x_j) \}_{(i, j)\in [N]\times [N]}$: 커널이 공분산 행렬을 생산함. $\text{Cov}(x_i, x_j) = k(x_i, x_j)$
> $i, j$가 가까울수록. $x$의 값이 딱 붙어있을수록 공분산의 절대값은 커져야 함.
>
> 공분산 행렬은 positive semi-definite여야 함. (GP의 정의에서 유도됨)

### **B. Kernel**

`-` 커널의 역할

* $k(x_i, x_j)$는 $f(x_i)$와 $f(x_j)$ 사이의 공분산을 의미
* 커널이 바뀌면 예측이 많이 달라짐

`-` RBF Kernel (Squared Exponential Kernel)

* 무한번 미분 가능한 함수, 매우 부드러운 개형의 커널
* input space에서 가까운 함수 값들은 높은 상관관계를 가지고, 멀리 떨어진 값은 uncorrelated

`-` Matern kernel

* $\nu$가 작을수록 더욱 러프한 샘플 함수를 제공

`-` Non-stationary Kernels

* Linear Kernel: $\sigma^2_0 + x^{\top}x$. 2차함수 개형. mean에서 벗어날수록 변동 심화
* Brownian motion: $\min(x_1, x_2)$. 미분 불가능.
* Periodic kernel: $\sin$함수 이용. 주기 표현
* Neural tangent kernel: DNN을 GP로 근사할 때 사용하는 일반적인 커널

`-` Combination of kernel for a new kernel

1. Sum: 단순합
2. Product: 단순곱
3. Convolution: $\int h(x, z)k(z, z')h(z', x)dzdz'$
> Convolution까지는 잘 활용할 필요가 없고, Sum과 Product만으로 충분함

### **C. Bayesian Prediction with GP**

* 비선형 함수에 대한 Prior로 사용
* Posterior는 데이터가 주어졌을 때의 Random Function

`-` Prior

* Likelihood의 함수에 적용. function space로 지정됨
* Regression: $y$는 평균만 관찰되는 것이 아니고, $\epsilon$이 같이 관측됨
* Classification: $y$의 확률을 계산하는 함수를 계산, 해당 함수에 Prior를 지정
> Regression은 Gaussian으로 한정지을 수 있으나, Classification은 Gaussian으로 가정할 수 없음. 다른 방법이 필요

`-` 다른 모델과의 관계

* 공분산 모델링은 GLM, Spline과 연관있음
* 무한한 수의 노드를 가지는 MLP는 GP로 근사됨

`-` Regression

* 다변량 정규 분포의 Closed Form 덕분에 Posterior 또한 Gaussian Process임
* Posterior의 평균은 $y_i$의 가중평균과 동일한 형태로 나타남 (어디까지나 최적화가 아닌 선형 변환의 형태니까...)
* 최적화를 하는 게 아니라 역행렬 계산. 이후 하이퍼파라미터 튜닝만 하면 됨
* 공분산 행렬의 역행렬을 계산해야되기 때문에 $O(N^3)$의 코스트 발생. 일부 샘플만으로 전체와 유사한 결과를 내도록 하는 방법이 요구됨 (Variational Bayes)

`-` Classification

* Classification에서는 $y$가 연속형이 아니기 때문에 Posterior는 가우시안이 아님. non-conjugate
* 가우시안이 아닌 애를 가우시안으로 억지로 만듦 (Gaussian Approximation)
> Laplace approximation: 봉우리가 하나이고, maximum을 가지는 어떠한 분포라도 정규분포로 근사할 수 있다. mode와 2차 미분을 이용
>
> Expectation propagation: KL divergence를 이용해서 정규분포에 가장 가까운 애를 찾음

## 4. Fully Factorized Gaussian

* VAE의 아이디어를 활용하여 베이지안 딥러닝을 구현하는 방법론
* 완전히 분해된(Fully Factorized) 독립적인 가우시안 분포

`-` 목적 및 기반

* DNN의 가중치 Posterior를 근사하여 베이지안 딥러닝을 구현
* Reparametrization trick과 ELBO를 학습에 이용
* MLP의 모든 가중치를 Prior로 지정

`-` Prior / Posterior Approximation

* 모든 가중치가 독립적인 표준 정규 분포를 따른다고 가정
* Posterior 또한 독립적인 정규 분포를 따른다고 가정 $(p(\theta~|~D) \approx q(\theta))$
> KL divergence를 이용하여 본래 분포와 근사 분포가 가까워지도록 최적화

`-` 알고리즘

* SGVB 기반 KL divergence 최소화
* Reparametrization trick

`-` 결과

* Credible Interval 제공
* mu + sigma x epsilon 형태를 유지만 할 수 있다면, t분포 같은 것으로도 확장 가능

## 5. Laplace Approximation

* 복잡한 분포를 정규 분포로 근사 (GP)
* 2차 테일러 급수 전개를 활용

`-` Posterior

* Maximum A Priori (MAP): Posterior의 Mode를 찾고, 여기서의 2차 헤시안 행렬 A를 계산
* 평균은 MAP 추정치(Mode)를 사용 $\theta^*$
* 헤시안은 곡률을 의미하고, 이 값이 클수록 sharp shape. 낮은 분산.
> 헤시안의 역행렬을 사용하면 헤시안의 역행렬과 분산이 같은 방향으로 움직임
>
> $- A^{-1}$를 covariance matrix로 사용
* 최종 사후 분포: $N(\theta~|~\theta^*, - A^{-1})$

`-` BIC / Evidence

* Evidence의 근사에 LA를 사용 (Bayesian Information Criterion과 동일)

$$-2 \log p(y) \approx - 2 \log p(y~|~\theta^*) + p \log n$$

> AIC보다 뒤쪽 term의 비중이 더 커짐. 데이터가 커질수록 모델 파라미터를 줄여야 한다. (뒤쪽 p는 그거임) Overfitting의 우려는 매우 줄어드나, 약간 Underfitting할 우려가 있음.
>
> BIC가 작을수록 Evidence가 큰 것과 비슷함.

따라서 Overfitting이 걱정된다면 베이지안 방법을 사용하라.

## 6. Neural Tangent Kernel

* DNN을 kernel machine으로 이해하기 위한 시도
* 커널은 그래디언트의 내적으로 정의됨: 초기값 설정 후 업데이트되며, 일정수준 안정화된 값(초기값)과 최종 업데이트값이 일치한다는 가정
* 실제 값과의 오차는 업데이트가 무한히 반복되었을 때 계속 줄어들면서 0으로 수렴

`-` 핵심사항

* Two hidden layer and infinite node (neural)를 가정
* 2차 테일러 전개를 활용하여 커널 만듦

`-` Gradient

* input projection $a_i$과 output projection $b_i$의 그래디언트를 계산
* 두 가지의 커널들이 $a, b$가 커짐에 따라 수렴함
* DNN이 주어지면 NTK를 계산할 수 있는 방식들이 존재함